In [ ]:
import gc
import argparse
import os
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import wandb
import torch.nn.functional as F
from exp.exp_main import Exp_Main
from utils.tools import dotdict
from pandas import DataFrame
from torch import nn, einsum
os.environ["WANDB_SILENT"] = "true"

In [ ]:
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

In [ ]:
#-----Baseline Transformers-----
for model in ['Autoformer', 'Informer']: 
    l = []    
    use_tf = False
    for data in [ 'covid', 'illness']:    
        if(data == 'illness'): 
            seq_len = 36 
            h = [24,36,48,60] 
            ftrs = 7 
            col = ['date','ILITOTAL','% WEIGHTED ILI','%UNWEIGHTED ILI','AGE 0-4','AGE 5-24','NUM. OF PROVIDERS','OT']
        elif(data == 'covid'): 
            seq_len = 36 
            h = [24,36,48,60]
            ftrs = 6 
            col = ['date','new_cases','new_deaths','icu_patients','hosp_patients','new_tests','new_vaccinations']
        for horizon in h: 
            project_name = ''+model+'_'+data+'_'+str(seq_len)+'_'+str(horizon)+''   
            sweep_configuration = {
                'method': 'random',
                'name': project_name,
                'metric': {
                    'goal': 'minimize', 
                    'name': 'mae'
                    },
                'parameters': {
                    'model':{'values':[model]},
                    'seq_len':{'values': [seq_len]},
                    'label_len':{'values': [18]},
                    'pred_len':{'values': [horizon]},
                    'e_layers':{'values': [2]},  
                    'd_layers':{'values': [2]},  
                    'd_model':{'values': [512]},
                    'n_heads':{'values': [6,8,10,11,12]},
                    'd_ff':{'values': [1024]},
                    'dropout':{'max': 0.4, 'min': 0.2},
                    'tf_ratio':{'max':0.7 , 'min': 0.3},
                    'lr': {'values':[0.0001]},
                    'units_layer1':{'values': [128,256,512]},
                    'units_layer2':{'values': [128,256,512]},
                    'act_function1':{'values': ['relu','elu']},
                    'act_function2':{'values': ['relu','elu']},
                    'hidden_dim':{'values': [128,256,512]},
                    'layer_dim':{'values': [1,2,3]}
                 }
                }

            sweep_id = wandb.sweep(sweep=sweep_configuration, project=project_name)
            def run():
                wandb.init(project=project_name)
                args = dotdict()
                args.model=wandb.config.model
                args.seq_len=wandb.config.seq_len
                args.label_len=wandb.config.label_len
                args.pred_len=wandb.config.pred_len
                args.e_layers=wandb.config.e_layers
                args.d_layers=wandb.config.d_layers
                args.d_model=wandb.config.d_model
                args.n_heads=wandb.config.n_heads
                args.d_ff = wandb.config.d_ff
                args.dropout = wandb.config.dropout
                args.learning_rate = wandb.config.lr
                args.tf_ratio = wandb.config.tf_ratio
                args.units_layer1 = wandb.config.units_layer1
                args.units_layer2 = wandb.config.units_layer2
                args.act_function1 = wandb.config.act_function1
                args.act_function2 = wandb.config.act_function2
                args.hidden_dim = wandb.config.hidden_dim
                args.layer_dim = wandb.config.layer_dim
                args.is_training=1
                args.root_path='./dataset/'+data+'/'
                args.data_path=data+'.csv'
                args.model_id='ili_36_'+str(horizon)
                args.data='custom'
                args.features='M'
                args.columns=col
                args.factor=3
                args.enc_in=ftrs
                args.dec_in=ftrs
                args.c_out=ftrs
                args.des='Exp'
                args.itr=1
                args.freq='h'
                args.checkpoints='./checkpoints/'
                args.distil = 'store_false'
                args.activation = 'relu'
                args.output_attention = False
                args.train_epochs = 10
                args.batch_size = 32
                args.patience = 3
                args.des = 'test'
                args.loss = 'mse'
                args.lradj = '6'
                args.use_amp = True
                args.gpu = 1
                args.use_multi_gpu = 'store_true'
                args.devices = '0'
                args.use_gpu = True
                args.inverse = False
                args.default_pe = True
                args.split = [0.68,0.2]
                args.moving_avg = 25
                args.project_name = project_name
                args.embed = 'timeF'
                args.use_tf = use_tf


                if args.use_gpu and args.use_multi_gpu:
                    args.dvices = args.devices.replace(' ', '')
                    device_ids = args.devices.split(',')
                    args.device_ids = [int(id_) for id_ in device_ids]
                    args.gpu = args.device_ids[0]

                Exp = Exp_Main

                for ii in range(args.itr):
                    # setting record of experiments
                    setting = 'id{}_m{}_d{}_f{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_dff{}_do{}_tf{}_lr{}_{}'.format(
                        args.model_id,
                        args.model,
                        args.data,
                        args.features,
                        args.seq_len,
                        args.label_len,
                        args.pred_len,
                        args.d_model,
                        args.n_heads,
                        args.e_layers,
                        args.d_layers,
                        args.d_ff,
                        args.dropout,
                        args.tf_ratio,
                        args.lr, 
                        ii)

                    exp = Exp(args)
                    exp.train(setting)
                    exp.test(setting)
            wandb.agent(sweep_id=sweep_id, function=run, count=10)
            api = wandb.Api()
            sweep = api.sweep(f"{'maldosari'}/{project_name}/{sweep_id}")
            best_run = sweep.best_run()
            l.append([best_run.config['model'],data, best_run.config['seq_len'],best_run.config['pred_len'],
                      best_run.summary_metrics['mse'],best_run.summary_metrics['mae'],best_run.summary_metrics['smape'],
                      best_run.config['e_layers'], best_run.config['d_layers'], best_run.config['d_model'], 
                      best_run.config['n_heads'], best_run.config['d_ff'], best_run.config['dropout'],
                      best_run.config['tf_ratio']])
    df = pd.DataFrame(l)
    df.columns = ['model','data','seq_len', 'pred_len','mse', 'mae','smape', 'e_layers', 'd_layers', 'd_model',
                         'n_heads', 'd_ff', 'dropout','tf_ratio']
    df.to_csv('tuning/'+model+'_results_all.csv',index=False)
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
#-----NAR-Transformer and AR-Transformers-----
for model in ['NAR_Transformer','AR_Transformer','AR_Transformer_TF']: 
    l = []
    if(model=='NAR_Transformer'):
        use_tf = False
        model_name = 'Transformer'
        label_len = 18
    elif(model=='AR_Transformer'):
        use_tf = False
        model_name = 'Transformer_TF'
        label_len = 18
    elif(model=='AR_Transformer_TF'):
        use_tf = True
        model_name = 'Transformer_TF'
        label_len = 18        
    for data in ['illness']:    
        if(data == 'illness'): 
            seq_len = 36 
            h = [24] 
            ftrs = 7 
            col = ['date','ILITOTAL','% WEIGHTED ILI','%UNWEIGHTED ILI','AGE 0-4','AGE 5-24','NUM. OF PROVIDERS','OT']
        elif(data == 'covid'): 
            seq_len = 36 
            h = [24,36,48,60]
            ftrs = 6 
            col = ['date','new_cases','new_deaths','icu_patients','hosp_patients','new_tests','new_vaccinations']
        for horizon in h: 
            project_name = ''+model+'_'+data+'_'+str(seq_len)+'_'+str(horizon)+''   
            sweep_configuration = {
                'method': 'random',
                'name': project_name,
                'metric': {
                    'goal': 'minimize', 
                    'name': 'mae'
                    },
                'parameters': {
                    'model':{'values':[model_name]},
                    'seq_len':{'values': [seq_len]},
                    'label_len':{'values': [label_len]},
                    'pred_len':{'values': [horizon]},
                    'e_layers':{'values': [2]},  
                    'd_layers':{'values': [2]},  
                    'd_model':{'values': [512]},
                    'n_heads':{'values': [6,8,10,11,12]},
                    'd_ff':{'values': [1024]},
                    'dropout':{'max': 0.4, 'min': 0.2},
                    'tf_ratio':{'max':0.7 , 'min': 0.3},
                    'lr': {'values':[0.0001]},
                    'units_layer1':{'values': [128,256,512]},
                    'units_layer2':{'values': [128,256,512]},
                    'act_function1':{'values': ['relu','elu']},
                    'act_function2':{'values': ['relu','elu']},
                    'hidden_dim':{'values': [128,256,512]},
                    'layer_dim':{'values': [1,2,3]}
                 }
                }

            sweep_id = wandb.sweep(sweep=sweep_configuration, project=project_name)
            def run():
                wandb.init(project=project_name)
                args = dotdict()
                args.model=wandb.config.model
                args.seq_len=wandb.config.seq_len
                args.label_len=wandb.config.label_len
                args.pred_len=wandb.config.pred_len
                args.e_layers=wandb.config.e_layers
                args.d_layers=wandb.config.d_layers
                args.d_model=wandb.config.d_model
                args.n_heads=wandb.config.n_heads
                args.d_ff = wandb.config.d_ff
                args.dropout = wandb.config.dropout
                args.learning_rate = wandb.config.lr
                args.tf_ratio = wandb.config.tf_ratio
                args.units_layer1 = wandb.config.units_layer1
                args.units_layer2 = wandb.config.units_layer2
                args.act_function1 = wandb.config.act_function1
                args.act_function2 = wandb.config.act_function2
                args.hidden_dim = wandb.config.hidden_dim
                args.layer_dim = wandb.config.layer_dim
                args.is_training=1
                args.root_path='./dataset/'+data+'/'
                args.data_path=data+'.csv'
                args.model_id='ili_36_'+str(horizon)
                args.data='custom'
                args.features='M'
                args.columns=col
                args.factor=3
                args.enc_in=ftrs
                args.dec_in=ftrs
                args.c_out=ftrs
                args.des='Exp'
                args.itr=1
                args.freq='h'
                args.checkpoints='./checkpoints/'
                args.distil = 'store_false'
                args.activation = 'relu'
                args.output_attention = False
                args.train_epochs = 10
                args.batch_size = 32
                args.patience = 3
                args.des = 'test'
                args.loss = 'mse'
                args.lradj = '6'
                args.use_amp = True
                args.gpu = 1
                args.use_multi_gpu = 'store_true'
                args.devices = '0'
                args.use_gpu = True
                args.inverse = False
                args.default_pe = True
                args.split = [0.68,0.2]
                args.moving_avg = 25
                args.project_name = project_name
                args.embed = 'timeF'
                args.use_tf = use_tf


                if args.use_gpu and args.use_multi_gpu:
                    args.dvices = args.devices.replace(' ', '')
                    device_ids = args.devices.split(',')
                    args.device_ids = [int(id_) for id_ in device_ids]
                    args.gpu = args.device_ids[0]

                Exp = Exp_Main

                for ii in range(args.itr):
                    # setting record of experiments
                    setting = 'id{}_m{}_d{}_f{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_dff{}_do{}_tf{}_lr{}_{}'.format(
                        args.model_id,
                        args.model,
                        args.data,
                        args.features,
                        args.seq_len,
                        args.label_len,
                        args.pred_len,
                        args.d_model,
                        args.n_heads,
                        args.e_layers,
                        args.d_layers,
                        args.d_ff,
                        args.dropout,
                        args.tf_ratio,
                        args.lr, 
                        ii)

                    exp = Exp(args)
                    exp.train(setting)
                    exp.test(setting)
            wandb.agent(sweep_id=sweep_id, function=run, count=10)
            api = wandb.Api()
            sweep = api.sweep(f"{'maldosari'}/{project_name}/{sweep_id}")
            best_run = sweep.best_run()
            l.append([best_run.config['model'],data, best_run.config['seq_len'],best_run.config['pred_len'],
                      best_run.summary_metrics['mse'],best_run.summary_metrics['mae'],best_run.summary_metrics['smape'],
                      best_run.config['e_layers'], best_run.config['d_layers'], best_run.config['d_model'], 
                      best_run.config['n_heads'], best_run.config['d_ff'], best_run.config['dropout'],
                      best_run.config['tf_ratio']])
    df = pd.DataFrame(l)
    df.columns = ['model','data','seq_len', 'pred_len','mse', 'mae','smape', 'e_layers', 'd_layers', 'd_model',
                         'n_heads', 'd_ff', 'dropout','tf_ratio']
    df.to_csv('tuning/'+model+'_results_all.csv',index=False)
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
#-----NAR-LSTM and AR-LSTM-----
for model in ['NAR_LSTM','AR_LSTM','AR_LSTM_TF']: 
    l = []
    if(model=='NAR_LSTM'):
        use_tf = False
        model_name = 'LSTM'
    elif(model=='AR_LSTM'):
        use_tf = False
        model_name = 'AR_LSTM_TF'
    elif(model=='R_LSTM_TF'):
        use_tf = True
        model_name = 'LSTMSeq2Seq'
        
    for data in [ 'covid', 'illness']:    
        if(data == 'illness'): 
            seq_len = 36 
            h = [24,36,48,60] 
            ftrs = 7 
            col = ['date','ILITOTAL','% WEIGHTED ILI','%UNWEIGHTED ILI','AGE 0-4','AGE 5-24','NUM. OF PROVIDERS','OT']
        elif(data == 'covid'): 
            seq_len = 36 
            h = [24,36,48,60]
            ftrs = 6 
            col = ['date','new_cases','new_deaths','icu_patients','hosp_patients','new_tests','new_vaccinations']
        for horizon in h: 
            project_name = ''+model+'_'+data+'_'+str(seq_len)+'_'+str(horizon)+''   
            sweep_configuration = {
                'method': 'random',
                'name': project_name,
                'metric': {
                    'goal': 'minimize', 
                    'name': 'mse'
                    },
                'parameters': {
                    'model':{'values':[model_name]},
                    'seq_len':{'values': [seq_len]},
                    'label_len':{'values': [label_len]},
                    'pred_len':{'values': [horizon]},
                    'e_layers':{'values': [2]},  
                    'd_layers':{'values': [2]},  
                    'd_model':{'values': [512]},
                    'n_heads':{'values': [6,8,10,11,12]},
                    'd_ff':{'values': [1024]},
                    'dropout':{'max': 0.4, 'min': 0.2},
                    'tf_ratio':{'max':0.7 , 'min': 0.3},
                    'lr': {'values':[0.0001]},
                    'units_layer1':{'values': [128,256,512]},
                    'units_layer2':{'values': [128,256,512]},
                    'act_function1':{'values': ['relu','elu']},
                    'act_function2':{'values': ['relu','elu']},
                    'hidden_dim':{'values': [128,256,512]},
                    'layer_dim':{'values': [1,2,3]}
                 }
                }

            sweep_id = wandb.sweep(sweep=sweep_configuration, project=project_name)
            def run():
                wandb.init(project=project_name)
                args = dotdict()
                args.model=wandb.config.model
                args.seq_len=wandb.config.seq_len
                args.label_len=wandb.config.label_len
                args.pred_len=wandb.config.pred_len
                args.e_layers=wandb.config.e_layers
                args.d_layers=wandb.config.d_layers
                args.d_model=wandb.config.d_model
                args.n_heads=wandb.config.n_heads
                args.d_ff = wandb.config.d_ff
                args.dropout = wandb.config.dropout
                args.learning_rate = wandb.config.lr
                args.tf_ratio = wandb.config.tf_ratio
                args.units_layer1 = wandb.config.units_layer1
                args.units_layer2 = wandb.config.units_layer2
                args.act_function1 = wandb.config.act_function1
                args.act_function2 = wandb.config.act_function2
                args.hidden_dim = wandb.config.hidden_dim
                args.layer_dim = wandb.config.layer_dim
                args.is_training=1
                args.root_path='./dataset/'+data+'/'
                args.data_path=data+'.csv'
                args.model_id='ili_36_'+str(horizon)
                args.data='custom'
                args.features='M'
                args.columns=col
                args.factor=3
                args.enc_in=ftrs
                args.dec_in=ftrs
                args.c_out=ftrs
                args.des='Exp'
                args.itr=1
                args.freq='h'
                args.checkpoints='./checkpoints/'
                args.distil = 'store_false'
                args.activation = 'relu'
                args.output_attention = False
                args.train_epochs = 10
                args.batch_size = 32
                args.patience = 3
                args.des = 'test'
                args.loss = 'mse'
                args.lradj = '6'
                args.use_amp = True
                args.gpu = 1
                args.use_multi_gpu = 'store_true'
                args.devices = '0'
                args.use_gpu = True
                args.inverse = False
                args.default_pe = True
                args.split = [0.68,0.2]
                args.moving_avg = 25
                args.project_name = project_name
                args.embed = 'timeF'
                args.use_tf = use_tf


                if args.use_gpu and args.use_multi_gpu:
                    args.dvices = args.devices.replace(' ', '')
                    device_ids = args.devices.split(',')
                    args.device_ids = [int(id_) for id_ in device_ids]
                    args.gpu = args.device_ids[0]

                Exp = Exp_Main

                for ii in range(args.itr):
                    # setting record of experiments
                    setting = 'id{}_m{}_d{}_f{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_dff{}_do{}_tf{}_lr{}_{}'.format(
                        args.model_id,
                        args.model,
                        args.data,
                        args.features,
                        args.seq_len,
                        args.label_len,
                        args.pred_len,
                        args.d_model,
                        args.n_heads,
                        args.e_layers,
                        args.d_layers,
                        args.d_ff,
                        args.dropout,
                        args.tf_ratio,
                        args.lr, 
                        ii)

                    exp = Exp(args)
                    exp.train(setting)
                    exp.test(setting)
            wandb.agent(sweep_id=sweep_id, function=run, count=10)
            api = wandb.Api()
            sweep = api.sweep(f"{'maldosari'}/{project_name}/{sweep_id}")
            best_run = sweep.best_run()
            l.append([best_run.config['model'],data, best_run.config['seq_len'],best_run.config['pred_len'],
                      best_run.summary_metrics['mse'],best_run.summary_metrics['mae'],best_run.summary_metrics['smape'],
                      best_run.config['e_layers'], best_run.config['d_layers'], best_run.config['d_model'], 
                      best_run.config['n_heads'], best_run.config['d_ff'], best_run.config['dropout'],
                      best_run.config['tf_ratio']])
    df = pd.DataFrame(l)
    df.columns = ['model','data','seq_len', 'pred_len','mse', 'mae','smape', 'e_layers', 'd_layers', 'd_model',
                         'n_heads', 'd_ff', 'dropout','tf_ratio']
    df.to_csv('tuning/'+model+'_results_all.csv',index=False)
    gc.collect()
    torch.cuda.empty_cache()